In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Discrete Shuffled Frog-leaping Algorithm for Social Influence Maximization

In [ ]:
# Installing the NetworkX Library to work with Network Graphs & their Visualizations
!pip install networkx

### Importing & Shaping Libraries, Modules & Datasets

In [ ]:
# The NetworkX Library is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks
import networkx as nx
# The Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language
import pandas as pd
# The Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python
import matplotlib.pyplot as plt
# The Random Module is a built-in Python Module to work with Random Numbers
import random
import copy

In [ ]:
# Generating a Dataframe from the books based on Game of Thrones
books = []
for i in range(0,5):
    books.append(pd.read_csv("/content/drive/MyDrive/DSFLA Datasets/BOOKS/Book{}.csv".format(i+1)))
all_books = pd.concat(books)
all_books.head()

,Source,Target,Type,weight,book
0,Addam-Marbrand,Jaime-Lannister,Undirected,3,1.0
1,Addam-Marbrand,Tywin-Lannister,Undirected,6,1.0
2,Aegon-I-Targaryen,Daenerys-Targaryen,Undirected,5,1.0
3,Aegon-I-Targaryen,Eddard-Stark,Undirected,4,1.0
4,Aemon-Targaryen-(Maester-Aemon),Alliser-Thorne,Undirected,4,1.0


In [ ]:
# Converting the Dataframe structure into an Edgelist to feed into the NetworkX Library function
edges = all_books.groupby(['Source','Target']).agg({'weight':'sum'}).reset_index()
edges.sort_values('weight', ascending = False)

,Source,Target,weight
1334,Eddard-Stark,Robert-Baratheon,334
2031,Jon-Snow,Samwell-Tarly,228
1965,Joffrey-Baratheon,Sansa-Stark,222
1972,Joffrey-Baratheon,Tyrion-Lannister,219
640,Bran-Stark,Hodor,209
...,...,...,...
1809,Jaime-Lannister,Joy-Hill,3
1805,Jaime-Lannister,Jon-Snow,3
1802,Jafer-Flowers,Jaremy-Rykker,3
1801,Jaehaerys-I-Targaryen,Maegor-I-Targaryen,3


In [ ]:
# Creating the NetworkX Graph using Pandas Edgelist
GOT = nx.from_pandas_edgelist(edges, source = 'Source', target = 'Target', edge_attr = 'weight')
print(nx.info(GOT))
# Dictionary storing the Degree of each node in the Graph
weighted_degrees = dict(nx.degree(GOT))

Graph with 796 nodes and 2823 edges


#### Creation & Calculation of Memeplexes for the Computation of Most Influential Nodes

In [ ]:
m = 5 # No. of memeplexes
n = 5 # No. of memes in each memeplex
k = 4 # No. of memetypes in each meme
itr = 2 # No. of iterations
F = m * n
population = []
for i in range(F):
    population.append([random.choice(list(weighted_degrees)) for i in range(k)])

# Printing the Randomly Generated Memes
for i in range(len(population)):
    print("Meme-{}".format(i+1), population[i])

Meme-1 ['Gorne', 'Maddy', 'Willas-Tyrell']
Meme-2 ['Dunsen', 'Richard-Horpe', 'Cutjack']
Meme-3 ['Jarman-Buckwell', 'Weasel', 'Tom-of-Sevenstreams']
Meme-4 ['Tion-Frey', 'Dobber', 'Rhaegar-Targaryen']
Meme-5 ['Fogo', 'Rugen', 'Ryk']
Meme-6 ['Galazza-Galare', 'Tregar', 'Lollys-Stokeworth']
Meme-7 ['Nan', 'Rossart', 'Nage']
Meme-8 ['Ramsay-Snow', 'Imry-Florent', 'High-Sparrow']
Meme-9 ['Renly-Baratheon', 'Elmar-Frey', 'Styr']
Meme-10 ['Tormund', 'Tattered-Prince', 'Grenn']
Meme-11 ['Lyle-Crakehall', 'Mikken', 'Jarl']
Meme-12 ['Grazdan-mo-Eraz', 'Shagga', 'Pate-(novice)']
Meme-13 ['Nan', 'Quhuru-Mo', 'Varys']
Meme-14 ['Drennan', 'Halder', 'Vayon-Poole']
Meme-15 ['Sweets', 'Hugh-Hungerford', 'Varamyr']
Meme-16 ['Kurz', 'Xhondo', 'Tomard']
Meme-17 ['Meryn-Trant', 'Jory-Cassel', 'Willam-Dustin']
Meme-18 ['Daeron-II-Targaryen', 'Brandon-Norrey', 'Stonesnake']
Meme-19 ['Godry-Farring', 'Joffrey-Baratheon', 'Eroeh']
Meme-20 ['Fogo', 'Quaro', 'Balon-Swann']
Meme-21 ['Cayn', 'Clydas', 'Tarber']
M

In [ ]:
def one_hop_area(meme):
    """
    Compute the One-Hop Area, i.e., the nodes adjacent to the nodes in the given meme
    Parameters:
        meme = Meme whose One Hop Area is to be calculated
    Returns:
        List of Nodes which are adjacent to atleast one node within a meme
    """
    temp = [] # Temporary list to store the One-Hop Area nodes of the meme
    for i in meme:
        temp.extend(list(GOT.neighbors(i)))
    return list(set(temp) - set(meme)) # Returning only unique nodes, to avoid Node Repetition

def two_hop_area(meme):
    """
    Compute the Two-Hop Area, i.e., the nodes adjacent to the One-Hop area nodes of the given meme
    Parameters:
        meme = Meme whose Two Hop Area is to be calculated
    Returns:
        List of Nodes which are adjacent to atleast one node within the One-Hop Area Nodes but not a part of the Meme
    """
    one_hop = one_hop_area(meme)
    temp = [] # Temporary list to store the Two-Hop Area nodes of the meme
    for i in one_hop:
        temp.extend(list(GOT.neighbors(i)))
    return list(set(temp) - set(meme) - set(one_hop)) # Returning only unique nodes, to avoid Node Repetition

def calc_pcm_prob(nodes):
    """
    Calculates the Constant Propogation Cascade Probability of the nodes of the given meme
    Parameters:
        nodes = List of Nodes whose Cascade Probability has to be calculated
    Returns:
        List of Probabilities where ith probability corresponds to the Cascade Probability of the ith node of the list of given nodes
    """
    prob = [] # Temporary List to store the corresponding Cascade Probability of the Nodes
    for i in nodes:
        prob.append(weighted_degrees[i] / GOT.number_of_nodes())
    return prob

def calc_edges(group1, group2):
    """
    Calculates the number of edges each Group-2 Node has within the nodes Group-1 & Group-2
    Parameters:
        group1 = First Group of Nodes
        group2 = Second Group of Nodes
    Returns:
        List of number of edges where ith entry corresponds to the number of edges Node-i of Group-2 has within nodes of Group-1 & Group-2
    """
    count = [] # Temporary storage to store the Number of edges corresponding each node
    for i in group2:
        edges = list(nx.edges(GOT, nbunch = [i]))
        temp = 0
        for i in edges:
            if (i[1] in group1) or (i[1] in group2):
                temp += 1
        count.append(temp)
    return count

def sum_pd(list1, list2):
    """
    Computes the sum-product of entries of two list
    Parameters:
        list1: A List of Floating-point Numbers
        list2: A List of Floating-point Numbers
    Returns:
        Sum of the Products of each corresponding Entry of the two lists
    """
    p = 0 # Temporary variable to store the Sum-Product of the two lists
    for i in range(len(list1)):
        p += (list1[i] * list2[i])
    return p

def calc_edge_prob(meme, one_hop):
    """
    Returns the sum of the Edges probabilities of the nodes with their One-Hop area nodes
    Parameters:
        meme = Meme whose nodes' edge probabilities have to be calculated
        one_hop = One-Hop Area nodes of the given meme
    Returns:
        Sum of the Edge Probabilities using the formula - 1 - Product(1 - Pij)
    """
    N = GOT.number_of_nodes() # Total Nodes in the Graph
    prob_sum = 0 # Temporary variable to store the sum of the Edge Probabilities
    for i in one_hop:
        prob_prod = 1
        for j in meme:
            pij = 0.01
            pij += ((GOT.degree(i) + GOT.degree(j)) / N)
            pij += (len(list(nx.common_neighbors(GOT, i, j))) / N)
            prob_prod *= (1 - pij)
        prob_sum += (1 - prob_prod)
    return prob_sum

def LIE(meme):
    """
    Calculates the Local Influence Spread Measure of the nodes of the meme
    Parameters:
        meme = Meme whose LIE value has to be calculated
    Returns:
        A Floating-point Number denoting the Local Influence Spread Measure
    """
    Ns1_S = one_hop_area(meme) # One-Hop area of the Meme
    Ns2_S = two_hop_area(meme) # Two-Hop Area of the Meme
    pu = calc_pcm_prob(Ns2_S)
    du = calc_edges(Ns1_S, Ns2_S)
    return k + ((1 + ((1 / len(Ns1_S)) * sum_pd(pu, du))) * calc_edge_prob(meme, Ns1_S))

def sort_memes(ppl_lie, ppl):
    """
    Sorts the Population of Memes on the basis of their LIE values
    Parameters:
        ppl_lie = LIE values of each corresponding meme
        ppl = Population of Memes
    Returns:
        Tuple of List denoting the Sorted LIE values and Population of Memes in descending order
    """
    result = sorted(list(zip(ppl_lie, ppl)), reverse = True)
    result = zip(*result)
    result = [list(tuple) for tuple in result]
    return result[0], result[1]

In [ ]:
population_LIE = [] # List to store the LIE values of all memes
for i in range(F):
    population_LIE.append(LIE(population[i]))
# Sorting the Memes on the basis of their LIE values
population_LIE, population = sort_memes(population_LIE, population)

In [ ]:
print("The Meme-wise sorted LIE values are as follows -")
for i in range(len(population)):
    print("Meme-{} :".format(i+1), population_LIE[i], population[i])

print("\nThe Top-{} Most Influential Node have an Influence Spread Measure of - : {}".format(k, population_LIE[0]))
print("The Top-{} Most Influential Nodes in the Network are :".format(k))
for i in range(len(population[0])):
    print(population[0][i])
print("The Average Influence Spread Measure using Degree Centrality is :", sum(population_LIE) / len(population_LIE))

The Meme-wise sorted LIE values are as follows -
Meme-1 : 29.288365280018922 ['Godry-Farring', 'Joffrey-Baratheon', 'Eroeh']
Meme-2 : 25.749061692141268 ['Meryn-Trant', 'Jory-Cassel', 'Willam-Dustin']
Meme-3 : 24.724725588351433 ['Drennan', 'Halder', 'Vayon-Poole']
Meme-4 : 24.712868905741885 ['Renly-Baratheon', 'Elmar-Frey', 'Styr']
Meme-5 : 24.261009010303397 ['Nan', 'Quhuru-Mo', 'Varys']
Meme-6 : 23.240348784585017 ['Ramsay-Snow', 'Imry-Florent', 'High-Sparrow']
Meme-7 : 22.817457788024672 ['Tion-Frey', 'Dobber', 'Rhaegar-Targaryen']
Meme-8 : 21.454259611029237 ['Fogo', 'Quaro', 'Balon-Swann']
Meme-9 : 21.224244954490626 ['Gorne', 'Maddy', 'Willas-Tyrell']
Meme-10 : 20.948082773644742 ['Guyard-Morrigen', 'Maekar-I-Targaryen', 'Tanda-Stokeworth']
Meme-11 : 20.881067782626857 ['Wyl-(guard)', 'Camarron', 'Beth-Cassel']
Meme-12 : 20.051245093419013 ['Dunsen', 'Richard-Horpe', 'Cutjack']
Meme-13 : 20.04881722980057 ['Kurz', 'Xhondo', 'Tomard']
Meme-14 : 19.438006023254438 ['Nan', 'Rossar

##### Memeplex Creation

In [ ]:
population = list(zip(population, population_LIE))

# Creating Memeplexes using Uniform Distribution of Memes based on their LIE values
memeplex = []
for i in range(m):
    meme = []
    for j in range(n):
        meme.append(population[i + (j * m)])
    memeplex.append(meme)

In [ ]:
print("The Generated Memeplexes are as follows -")
for i in range(m):
    print("\nMemeplex - {} :".format(i+1))
    for j in range(n):
        print(memeplex[i][j][1], memeplex[i][j][0])

The Generated Memeplexes are as follows -

Memeplex - 1 :
29.288365280018922 ['Godry-Farring', 'Joffrey-Baratheon', 'Eroeh']
23.240348784585017 ['Ramsay-Snow', 'Imry-Florent', 'High-Sparrow']
20.881067782626857 ['Wyl-(guard)', 'Camarron', 'Beth-Cassel']
18.983834510420408 ['Galazza-Galare', 'Tregar', 'Lollys-Stokeworth']
14.072406675227302 ['Jarman-Buckwell', 'Weasel', 'Tom-of-Sevenstreams']

Memeplex - 2 :
25.749061692141268 ['Meryn-Trant', 'Jory-Cassel', 'Willam-Dustin']
22.817457788024672 ['Tion-Frey', 'Dobber', 'Rhaegar-Targaryen']
20.051245093419013 ['Dunsen', 'Richard-Horpe', 'Cutjack']
18.908711302781217 ['Cayn', 'Clydas', 'Tarber']
12.378697489945827 ['Daeron-II-Targaryen', 'Brandon-Norrey', 'Stonesnake']

Memeplex - 3 :
24.724725588351433 ['Drennan', 'Halder', 'Vayon-Poole']
21.454259611029237 ['Fogo', 'Quaro', 'Balon-Swann']
20.04881722980057 ['Kurz', 'Xhondo', 'Tomard']
18.602948699192623 ['Dunsen', 'Talea', 'Rhaegar-Frey']
11.988539630091994 ['Grazdan-mo-Eraz', 'Shagga', 'P

In [ ]:
def sort_nodes(nodes, centrality):
    """
    Sorts the Nodes in the given list of nodes based on the corresponding Centrality Measure of each node
    Parameters:
        nodes = List of nodes which has to be sorted
        centrality = List of Centrality Measures corresponding to each node in the list
    Returns:
        List of nodes sorted on the basis of their Centrality Measure in descending order
    """
    result = sorted(list(zip(centrality, nodes)), reverse = True)
    result = zip(*result)
    result = [list(tuple) for tuple in result]
    return result[1]

def sort_degree_centrality(nodes):
    """
    Calculates the Degree Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def sort_eigenvector_centrality(nodes):
    """
    Calculates the Eigenvector Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def sort_betweenness_centrality(nodes):
    """
    Calculates the Betweenness Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def sort_closeness_centrality(nodes):
    """
    Calculates the Closeness Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def sort_katz_centrality(nodes):
    """
    Calculates the Katz Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def sort_percolation_centrality(nodes):
    """
    Calculates the Percolation Centrality of each Node in the given list of nodes and returns a descendingly sorted list of nodes
    Parameters:
        nodes = List of nodes to be sorted on the basis of the Centrality Measure
    Returns:
        List of Nodes sorted in descending order on the basis of the Centrality Measure
    """
    centrality = nx.degree_centrality(GOT)
    cen_measure = []
    for i in nodes:
        cen_measure.append(centrality[i])
    return sort_nodes(nodes, cen_measure)

def LDR(meme, flag):
    """
    Calculates the Local-Replacement group for the given meme for the Evolutionary Algorithm
    Parameters:
        meme = Meme whose Local-Replacement group has to be calculated
        flag = an integer (1-5) to denote the Centrality Measure to be used for the Evolution Process
                1 - Degree Centrality
                2 - Eigenvector Centrality
                3 - Betweenness Centrality
                4 - Closeness Centrality
                5 - Katz Centrality
                6 - Percolation Centrality
    Returns:
        A List of Top-Centrality nodes equal to the length of the passed meme
    """
    new_meme = [] # New Meme to improve the worst Meme(Frog) w.r.t. the Local/Global Best Meme (Frog)
    N1 = []
    for i in meme: # For each Memetype in the Meme
        N1.extend(list(GOT.neighbors(i)))
    N1 = list(set(N1))
    # Sort Function to sort the One Hop Neighbors of the memetype based on the basis of Centrality Metric
    if flag == 1:
        SN1 = sort_degree_centrality(N1)
    elif flag == 2:
        SN1 = sort_eigenvector_centrality(N1)
    elif flag == 3:
        SN1 = sort_betweenness_centrality(N1)
    elif flag == 4:
        SN1 = sort_closeness_centrality(N1)
    elif flag == 5:
        SN1 = sort_katz_centrality(N1)
    else:
        SN1 = sort_percolation_centrality(N1)
    for i in range(k):
        new_meme.append(SN1[i])
    return new_meme

def LDR_random():
    """
    Calculates a Random Meme for Local-Replacement
    Parameters:
        None
    Returns:
        A List of randomly selected nodes equal to the length of the passed meme
    """
    return [random.choice(list(weighted_degrees)) for i in range(k)]

def create_sub_memeplex(memeplex):
    """
    Generates a Sub-memeplex for a given Memeplex
    Parameters:
        memeplex = Memeplex whose Sub-Memeplex has to be calculated
    Returns:
        A Sub-Memeplex
        A List of positions(indices) to locate the corresponding memes within the memeplex
    """
    sub = []
    for i in range(n):
        sub.append(random.randint(0, n-1))
    sub = list(set(sub))
    sub_memeplex = [memeplex[i] for i in sub]
    return sub_memeplex, sub

In [ ]:
degree_mplx = copy.deepcopy(memeplex)
eigen_mplx = copy.deepcopy(memeplex)
between_mplx = copy.deepcopy(memeplex)
close_mplx = copy.deepcopy(memeplex)
katz_mplx = copy.deepcopy(memeplex)
perc_mplx = copy.deepcopy(memeplex)

### DSFLA using Degree Centrality for Evolutionary process

In [ ]:
for x in range(itr):
    Px = degree_mplx[0][0][0]
    Px_LIE = degree_mplx[0][0][1]
    for i in range(1, m):
        if degree_mplx[i][0][1] < Px_LIE:
            Px = degree_mplx[i][0][0]
            Px_LIE = degree_mplx[i][0][1]
    temp2 = LDR(Px, 1)
    
    for i in degree_mplx:
        sub_degree_mplx, pos = create_sub_memeplex(i)
        Pb, Pb_LIE = sub_degree_mplx[0][0], sub_degree_mplx[0][1]
        Pw, Pw_LIE = sub_degree_mplx[-1][0], sub_degree_mplx[-1][1]
        temp1 = LDR(Pb, 1)
        temp1 = LDR(Pb, 1)
        if LIE(temp1) > Pw_LIE:
            Pw = temp1
        elif LIE(temp2) > Pw_LIE:
            Pw = temp2
        elif LIE(temp3) > Pw_LIE:
            Pw = temp3
        else:
            Pw = LDR_random()
        Pw_LIE = LIE(Pw)
        sub_degree_mplx[-1] = [Pw, Pw_LIE]

        j = 0
        for p in pos:
            i[p] = sub_degree_mplx[j]
            j += 1
        
        meme, meme_LIE = [], []
        for p in i:
            meme.append(p[0])
            meme_LIE.append(p[1])
        meme_LIE, meme = sort_memes(meme_LIE, meme)

        for p in range(n):
            i[p] = [meme[p], meme_LIE[p]]

In [ ]:
population = []
population_LIE = []
for i in degree_mplx:
    for j in i:
        population.append(j[0])
        population_LIE.append(j[1])
population_LIE, population = sort_memes(population_LIE, population)
    
print("The Top-{} Most Influential Node have an Influence Spread Measure of - : {}".format(k, population_LIE[0]))
print("The Top-{} Most Influential Nodes in the Network are :".format(k))
for i in range(len(population[0])):
    print(population[0][i])
print("The Average Influence Spread Measure using Degree Centrality is :", sum(population_LIE) / len(population_LIE))

The Top-3 Most Influential Node have an Influence Spread Measure of - : 125.53717348343791
The Top-3 Most Influential Nodes in the Network are :
Tyrion-Lannister
Jon-Snow
Jaime-Lannister
The Average Influence Spread Measure using Degree Centrality is : 41.05833042406098


### DSFLA using Eigenvector Centrality for Evolutionary process

In [ ]:
for x in range(itr):
    Px = eigen_mplx[0][0][0]
    Px_LIE = eigen_mplx[0][0][1]
    for i in range(1, m):
        if eigen_mplx[i][0][1] < Px_LIE:
            Px = eigen_mplx[i][0][0]
            Px_LIE = eigen_mplx[i][0][1]
    temp2 = LDR(Px, 2)
    
    for i in eigen_mplx:
        sub_eigen_mplx, pos = create_sub_memeplex(i)
        Pb, Pb_LIE = sub_eigen_mplx[0][0], sub_eigen_mplx[0][1]
        Pw, Pw_LIE = sub_eigen_mplx[-1][0], sub_eigen_mplx[-1][1]
        temp1 = LDR(Pb, 2)
        temp1 = LDR(Pb, 2)
        if LIE(temp1) > Pw_LIE:
            Pw = temp1
        elif LIE(temp2) > Pw_LIE:
            Pw = temp2
        elif LIE(temp3) > Pw_LIE:
            Pw = temp3
        else:
            Pw = LDR_random()
        Pw_LIE = LIE(Pw)
        sub_eigen_mplx[-1] = [Pw, Pw_LIE]

        j = 0
        for p in pos:
            i[p] = sub_eigen_mplx[j]
            j += 1
        
        meme, meme_LIE = [], []
        for p in i:
            meme.append(p[0])
            meme_LIE.append(p[1])
        meme_LIE, meme = sort_memes(meme_LIE, meme)

        for p in range(n):
            i[p] = [meme[p], meme_LIE[p]]

In [ ]:
population = []
population_LIE = []
for i in eigen_mplx:
    for j in i:
        population.append(j[0])
        population_LIE.append(j[1])
population_LIE, population = sort_memes(population_LIE, population)
    
print("The Top-{} Most Influential Node have an Influence Spread Measure of - : {}".format(k, population_LIE[0]))
print("The Top-{} Most Influential Nodes in the Network are :".format(k))
for i in range(len(population[0])):
    print(population[0][i])
print("The Average Influence Spread Measure using Eigenvector Centrality is :", sum(population_LIE) / len(population_LIE))

The Top-3 Most Influential Node have an Influence Spread Measure of - : 125.53717348343791
The Top-3 Most Influential Nodes in the Network are :
Tyrion-Lannister
Jon-Snow
Jaime-Lannister
The Average Influence Spread Measure using Eigenvector Centrality is : 38.11869200562125


### DSFLA using Betweenness Centrality for Evolutionary process

In [ ]:
for x in range(itr):
    Px = between_mplx[0][0][0]
    Px_LIE = between_mplx[0][0][1]
    for i in range(1, m):
        if between_mplx[i][0][1] < Px_LIE:
            Px = between_mplx[i][0][0]
            Px_LIE = between_mplx[i][0][1]
    temp2 = LDR(Px, 3)
    
    for i in between_mplx:
        sub_between_mplx, pos = create_sub_memeplex(i)
        Pb, Pb_LIE = sub_between_mplx[0][0], sub_between_mplx[0][1]
        Pw, Pw_LIE = sub_between_mplx[-1][0], sub_between_mplx[-1][1]
        temp1 = LDR(Pb, 3)
        temp1 = LDR(Pb, 3)
        if LIE(temp1) > Pw_LIE:
            Pw = temp1
        elif LIE(temp2) > Pw_LIE:
            Pw = temp2
        elif LIE(temp3) > Pw_LIE:
            Pw = temp3
        else:
            Pw = LDR_random()
        Pw_LIE = LIE(Pw)
        sub_between_mplx[-1] = [Pw, Pw_LIE]

        j = 0
        for p in pos:
            i[p] = sub_between_mplx[j]
            j += 1
        
        meme, meme_LIE = [], []
        for p in i:
            meme.append(p[0])
            meme_LIE.append(p[1])
        meme_LIE, meme = sort_memes(meme_LIE, meme)

        for p in range(n):
            i[p] = [meme[p], meme_LIE[p]]

In [ ]:
population = []
population_LIE = []
for i in between_mplx:
    for j in i:
        population.append(j[0])
        population_LIE.append(j[1])
population_LIE, population = sort_memes(population_LIE, population)
    
print("The Top-{} Most Influential Node have an Influence Spread Measure of - : {}".format(k, population_LIE[0]))
print("The Top-{} Most Influential Nodes in the Network are :".format(k))
for i in range(len(population[0])):
    print(population[0][i])
print("The Average Influence Spread Measure using Betweenness Centrality is :", sum(population_LIE) / len(population_LIE))

The Top-3 Most Influential Node have an Influence Spread Measure of - : 125.53717348343791
The Top-3 Most Influential Nodes in the Network are :
Tyrion-Lannister
Jon-Snow
Jaime-Lannister
The Average Influence Spread Measure using Betweenness Centrality is : 41.08357213426147


### DSFLA using Closeness Centrality for Evolutionary process

In [ ]:
for x in range(itr):
    Px = close_mplx[0][0][0]
    Px_LIE = close_mplx[0][0][1]
    for i in range(1, m):
        if close_mplx[i][0][1] < Px_LIE:
            Px = close_mplx[i][0][0]
            Px_LIE = close_mplx[i][0][1]
    temp2 = LDR(Px, 4)
    
    for i in close_mplx:
        sub_close_mplx, pos = create_sub_memeplex(i)
        Pb, Pb_LIE = sub_close_mplx[0][0], sub_close_mplx[0][1]
        Pw, Pw_LIE = sub_close_mplx[-1][0], sub_close_mplx[-1][1]
        temp1 = LDR(Pb, 4)
        temp1 = LDR(Pb, 4)
        if LIE(temp1) > Pw_LIE:
            Pw = temp1
        elif LIE(temp2) > Pw_LIE:
            Pw = temp2
        elif LIE(temp3) > Pw_LIE:
            Pw = temp3
        else:
            Pw = LDR_random()
        Pw_LIE = LIE(Pw)
        sub_close_mplx[-1] = [Pw, Pw_LIE]

        j = 0
        for p in pos:
            i[p] = sub_close_mplx[j]
            j += 1
        
        meme, meme_LIE = [], []
        for p in i:
            meme.append(p[0])
            meme_LIE.append(p[1])
        meme_LIE, meme = sort_memes(meme_LIE, meme)

        for p in range(n):
            i[p] = [meme[p], meme_LIE[p]]

In [ ]:
population = []
population_LIE = []
for i in close_mplx:
    for j in i:
        population.append(j[0])
        population_LIE.append(j[1])
population_LIE, population = sort_memes(population_LIE, population)
    
print("The Top-{} Most Influential Node have an Influence Spread Measure of - : {}".format(k, population_LIE[0]))
print("The Top-{} Most Influential Nodes in the Network are :".format(k))
for i in range(len(population[0])):
    print(population[0][i])
print("The Average Influence Spread Measure using Closeness Centrality is :", sum(population_LIE) / len(population_LIE))

The Top-3 Most Influential Node have an Influence Spread Measure of - : 125.53717348343791
The Top-3 Most Influential Nodes in the Network are :
Tyrion-Lannister
Jon-Snow
Jaime-Lannister
The Average Influence Spread Measure using Closeness Centrality is : 41.224738663543995


### DSFLA using Katz Centrality for Evolutionary process

In [ ]:
for x in range(itr):
    Px = katz_mplx[0][0][0]
    Px_LIE = katz_mplx[0][0][1]
    for i in range(1, m):
        if katz_mplx[i][0][1] < Px_LIE:
            Px = katz_mplx[i][0][0]
            Px_LIE = katz_mplx[i][0][1]
    temp2 = LDR(Px, 5)
    
    for i in katz_mplx:
        sub_katz_mplx, pos = create_sub_memeplex(i)
        Pb, Pb_LIE = sub_katz_mplx[0][0], sub_katz_mplx[0][1]
        Pw, Pw_LIE = sub_katz_mplx[-1][0], sub_katz_mplx[-1][1]
        temp1 = LDR(Pb, 5)
        temp1 = LDR(Pb, 5)
        if LIE(temp1) > Pw_LIE:
            Pw = temp1
        elif LIE(temp2) > Pw_LIE:
            Pw = temp2
        elif LIE(temp3) > Pw_LIE:
            Pw = temp3
        else:
            Pw = LDR_random()
        Pw_LIE = LIE(Pw)
        sub_katz_mplx[-1] = [Pw, Pw_LIE]

        j = 0
        for p in pos:
            i[p] = sub_katz_mplx[j]
            j += 1
        
        meme, meme_LIE = [], []
        for p in i:
            meme.append(p[0])
            meme_LIE.append(p[1])
        meme_LIE, meme = sort_memes(meme_LIE, meme)

        for p in range(n):
            i[p] = [meme[p], meme_LIE[p]]

In [ ]:
population = []
population_LIE = []
for i in katz_mplx:
    for j in i:
        population.append(j[0])
        population_LIE.append(j[1])
population_LIE, population = sort_memes(population_LIE, population)
    
print("The Top-{} Most Influential Node have an Influence Spread Measure of - : {}".format(k, population_LIE[0]))
print("The Top-{} Most Influential Nodes in the Network are :".format(k))
for i in range(len(population[0])):
    print(population[0][i])
print("The Average Influence Spread Measure using Katz Centrality is :", sum(population_LIE) / len(population_LIE))

The Top-3 Most Influential Node have an Influence Spread Measure of - : 125.53717348343791
The Top-3 Most Influential Nodes in the Network are :
Tyrion-Lannister
Jon-Snow
Jaime-Lannister
The Average Influence Spread Measure using Katz Centrality is : 38.98245891163142


### DSFLA using Percolation Centrality for Evolutionary process

In [ ]:
for x in range(itr):
    Px = perc_mplx[0][0][0]
    Px_LIE = perc_mplx[0][0][1]
    for i in range(1, m):
        if perc_mplx[i][0][1] < Px_LIE:
            Px = perc_mplx[i][0][0]
            Px_LIE = perc_mplx[i][0][1]
    temp2 = LDR(Px, 6)
    
    for i in perc_mplx:
        sub_perc_mplx, pos = create_sub_memeplex(i)
        Pb, Pb_LIE = sub_perc_mplx[0][0], sub_perc_mplx[0][1]
        Pw, Pw_LIE = sub_perc_mplx[-1][0], sub_perc_mplx[-1][1]
        temp1 = LDR(Pb, 6)
        temp1 = LDR(Pb, 6)
        if LIE(temp1) > Pw_LIE:
            Pw = temp1
        elif LIE(temp2) > Pw_LIE:
            Pw = temp2
        elif LIE(temp3) > Pw_LIE:
            Pw = temp3
        else:
            Pw = LDR_random()
        Pw_LIE = LIE(Pw)
        sub_perc_mplx[-1] = [Pw, Pw_LIE]

        j = 0
        for p in pos:
            i[p] = sub_perc_mplx[j]
            j += 1
        
        meme, meme_LIE = [], []
        for p in i:
            meme.append(p[0])
            meme_LIE.append(p[1])
        meme_LIE, meme = sort_memes(meme_LIE, meme)

        for p in range(n):
            i[p] = [meme[p], meme_LIE[p]]

In [ ]:
population = []
population_LIE = []
for i in perc_mplx:
    for j in i:
        population.append(j[0])
        population_LIE.append(j[1])
population_LIE, population = sort_memes(population_LIE, population)
    
print("The Top-{} Most Influential Node have an Influence Spread Measure of - : {}".format(k, population_LIE[0]))
print("The Top-{} Most Influential Nodes in the Network are :".format(k))
for i in range(len(population[0])):
    print(population[0][i])
print("The Average Influence Spread Measure using Percolation Centrality is :", sum(population_LIE) / len(population_LIE))

The Top-3 Most Influential Node have an Influence Spread Measure of - : 125.53717348343791
The Top-3 Most Influential Nodes in the Network are :
Tyrion-Lannister
Jon-Snow
Jaime-Lannister
The Average Influence Spread Measure using Percolation Centrality is : 37.41237112359008
